# 🛡️ SHIELD Analytics System - KPI Analysis
## Análise Completa de Dados e Métricas de Negócio

**Data:** 24 de dezembro de 2025  
**Versão:** 2.4

Este notebook demonstra a estrutura completa do sistema SHIELD Analytics, incluindo:
- Carregamento e exploração dos dados da camada Gold
- Demonstração de todos os 11 KPIs disponíveis
- Visualizações interativas com Plotly
- Análises comparativas entre bancos, regiões e produtos
- Cálculos de métricas de negócio

## 📚 Índice de Conteúdo

1. [Importação e Setup](#setup)
2. [Carregamento de Dados](#dados)
3. [Exploração Estrutural](#exploração)
4. [KPI 1: Volume Total](#kpi1)
5. [KPI 2: Market Share](#kpi2)
6. [KPI 3: Contratos Ativos](#kpi3)
7. [KPI 4: Nível de Risco](#kpi4)
8. [KPI 5: Volume por Tempo](#kpi5)
9. [KPI 6: Share por Produto](#kpi6)
10. [KPI 7: Análise Regional](#kpi7)
11. [KPI 8: Risco vs Volume](#kpi8)
12. [Análises Comparativas](#comparativas)
13. [Insights e Conclusões](#insights)

---

# 1. Importação e Setup

In [8]:
# Importar bibliotecas necessárias
import pandas as pd
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# Configurar display de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')

print("✅ Bibliotecas importadas com sucesso!")
print(f"📦 Pandas: {pd.__version__}")


✅ Bibliotecas importadas com sucesso!
📦 Pandas: 2.2.3


In [9]:
# Classe ShieldMetrics - Definir localmente para uso no notebook
class ShieldMetrics:
    """Classe para cálculo de métricas específicas do Banco Shield"""
    
    @staticmethod
    def volume_total(df):
        """Soma de todos os valores financiados"""
        return df['financed_amount'].sum()
    
    @staticmethod
    def options(df, column):
        """Retorna valores únicos de uma coluna"""
        return df[column].unique()
    
    @staticmethod
    def market_share(df, banco_ref='Banco Shield'):
        """Percentual de participação de mercado"""
        total = df['financed_amount'].sum()
        if total == 0:
            return 0
        shield = df[df['bank_name'] == banco_ref]['financed_amount'].sum()
        return (shield / total) * 100
    
    @staticmethod
    def contratos_ativos(df):
        """Total de contratos únicos"""
        return len(df['contract_id'])
    
    @staticmethod
    def risco_medio(df):
        """Score de risco médio em percentual"""
        return df['risk_score'].mean() * 100
    
    @staticmethod
    def volume_tempo(df):
        """Volume por período e banco"""
        df_temp = df.copy()
        df_temp['ano_mes'] = df_temp['ano_mes'].apply(lambda x: f"{x[:4]}-{x[4:6]}")
        return (
            df_temp.groupby(['ano_mes', 'bank_name'])['financed_amount']
            .sum()
            .reset_index()
        )
    
    @staticmethod
    def share_por_produto(df):
        """Volume por produto e banco"""
        return (
            df.groupby(['product_name', 'bank_name'])['financed_amount']
            .sum()
            .reset_index()
        )
    
    @staticmethod
    def group_region(df):
        """Volume por região e banco"""
        return (
            df.groupby(['location_name', 'bank_name'])['financed_amount']
            .sum()
            .reset_index()
        )
    
    @staticmethod
    def risco_volume_contratos(df):
        """Agregação: volume, risco e contratos por produto"""
        return df.groupby(['product_name', 'bank_name']).agg({
            'financed_amount': 'sum', 
            'risk_score': 'mean',
            'contract_id': 'count'
        }).reset_index()

print("✅ Classe ShieldMetrics definida com sucesso!")


✅ Classe ShieldMetrics definida com sucesso!


---

# 2. Carregamento e Exploração de Dados

## 2.1 Carregar dados do Parquet Gold

In [10]:
# Definir caminho para dados Gold

from pathlib import Path

PROJECT_ROOT = Path().resolve().parent

DATA_DIR = PROJECT_ROOT / "medalion-architeture"
GOLD_DIR   = DATA_DIR / "03-gold-enriched"

gold_path = GOLD_DIR / "gold_fato_contratos_detalhada.parquet"


# Carregar dados
if gold_path.exists():
    df = pd.read_parquet(gold_path)
    print(f"✅ Dados carregados com sucesso!")
    print(f"📊 Dimensões: {df.shape[0]:,} linhas × {df.shape[1]} colunas")
    print(f"💾 Tamanho em memória: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
else:
    print(f"❌ Arquivo não encontrado em: {gold_path}")


✅ Dados carregados com sucesso!
📊 Dimensões: 5,945 linhas × 21 colunas
💾 Tamanho em memória: 3.09 MB


## 2.2 Primeiras linhas dos dados

In [11]:
df.head(10)

,contract_id,ano_mes,dim_banks_id,bank_name,is_competitor,product_id,product_name,category_name,prazo,taxa_base,location_id,location_name,macro_region,regional_risk_factor,units,financed_amount,outstanding_balance,dpd_30,risk_score,pct_amortized,is_high_risk
0,C202501-BA-000001,202501,1,Banco Shield,0,1013,Seguro Multiverso,Consórcio,12,0.25,509,São Paulo,Brasil,1.22,1,"24,668.11","19,184.31",0,0.08,0.78,0
1,C202501-BA-000002,202501,1,Banco Shield,0,1008,Plano Família Barton,Empréstimo,48,0.37,508,Rio de Janeiro,Brasil,1.07,1,"15,908.45","16,095.09",0,0.11,1.01,0
2,C202501-BA-000003,202501,1,Banco Shield,0,1009,Conta Digital SHIELD,Conta,24,0.33,508,Rio de Janeiro,Brasil,1.07,1,0.00,0.00,0,0.09,NaN,0
3,C202501-BA-000004,202501,1,Banco Shield,0,1003,Consórcio Stark,Empréstimo,48,0.15,501,Nova York,Norte América,1.03,1,"13,372.75","9,564.60",0,0.09,0.72,0
4,C202501-BA-000005,202501,1,Banco Shield,0,1009,Conta Digital SHIELD,Conta,24,0.33,515,Brasília,Brasil,0.92,1,0.00,0.00,0,0.08,NaN,0
5,C202501-BA-000006,202501,1,Banco Shield,0,1004,Seguro Asgard,Conta,24,0.29,514,Porto Alegre,Brasil,0.97,1,0.00,0.00,0,0.05,NaN,0
6,C202501-BA-000007,202501,1,Banco Shield,0,1019,Seguro Sokovia,Investimentos,36,0.30,506,Knowhere,Galáxia,1.22,1,"5,512.85","3,849.61",0,0.13,0.70,0
7,C202501-BA-000008,202501,1,Banco Shield,0,1020,Crédito Kamar-Taj,Consórcio,48,0.21,514,Porto Alegre,Brasil,0.97,1,"30,785.01","22,240.92",0,0.11,0.72,0
8,C202501-BA-000009,202501,1,Banco Shield,0,1020,Crédito Kamar-Taj,Consórcio,48,0.21,513,Recife,Brasil,0.95,1,"2,474.09","2,534.23",0,0.09,1.02,0
9,C202501-BA-000010,202501,1,Banco Shield,0,1009,Conta Digital SHIELD,Conta,24,0.33,504,Sokovia,Europa,1.24,1,0.00,0.00,0,0.10,NaN,0


## 2.3 Informações sobre as colunas

In [12]:
# Informações sobre tipos de dados
print("=" * 80)
print("INFORMAÇÕES SOBRE COLUNAS")
print("=" * 80)
print(f"\n{'Coluna':<30} {'Tipo':<15} {'Nulos':<10} {'Valores Únicos':<15}")
print("-" * 80)

for col in df.columns:
    dtype = df[col].dtype
    nulls = df[col].isnull().sum()
    unique = df[col].nunique()
    print(f"{col:<30} {str(dtype):<15} {nulls:<10} {unique:<15}")


INFORMAÇÕES SOBRE COLUNAS

Coluna                         Tipo            Nulos      Valores Únicos 
--------------------------------------------------------------------------------
contract_id                    object          0          5929           
ano_mes                        object          0          13             
dim_banks_id                   int64           0          2              
bank_name                      object          0          2              
is_competitor                  int32           0          2              
product_id                     int64           0          20             
product_name                   object          0          20             
category_name                  object          0          6              
prazo                          int64           0          5              
taxa_base                      float64         0          20             
location_id                    int64           0          15             
loca

## 2.4 Estatísticas descritivas das métricas financeiras

In [13]:
df[['financed_amount', 'outstanding_balance', 'units', 'dpd_30', 'risk_score']].describe()

,financed_amount,outstanding_balance,units,dpd_30,risk_score
count,"5,945.00","5,945.00","5,945.00","5,945.00","5,945.00"
mean,"14,362.64","11,815.84",1.00,0.94,0.11
std,"16,874.98","14,105.99",0.00,5.03,0.04
min,0.00,0.00,1.00,0.00,0.01
25%,722.05,583.35,1.00,0.00,0.08
50%,"6,775.98","5,638.42",1.00,0.00,0.11
75%,"23,991.82","18,939.24",1.00,0.00,0.14
max,"85,424.12","77,274.12",1.00,90.00,0.28


---

# 3. KPI 1: Volume Total

**Descrição:** Soma de todos os valores financiados em um período.

**Fórmula:**  
$$\text{Volume Total} = \sum \text{financed\_amount}$$

**Parâmetros:** DataFrame com dados filtrados  
**Retorno:** float - Soma total em R$

In [14]:
# Cálculo do Volume Total
vol_total = ShieldMetrics.volume_total(df)

print("=" * 80)
print("KPI 1: VOLUME TOTAL")
print("=" * 80)
print(f"\nVolume Total: R$ {vol_total:,.2f}")
print(f"Volume Total: R$ {vol_total/1e6:.2f} Milhões")
print(f"Volume Total: R$ {vol_total/1e9:.2f} Bilhões")

# Breakdown por banco
print("\n" + "-" * 80)
print("VOLUME POR BANCO:")
print("-" * 80)
vol_por_banco = df.groupby('bank_name')['financed_amount'].agg(['sum', 'count'])
vol_por_banco.columns = ['Volume Total', 'Qtd Registros']
vol_por_banco['% do Total'] = (vol_por_banco['Volume Total'] / vol_total * 100)
print(vol_por_banco)


KPI 1: VOLUME TOTAL

Volume Total: R$ 85,385,907.55
Volume Total: R$ 85.39 Milhões
Volume Total: R$ 0.09 Bilhões

--------------------------------------------------------------------------------
VOLUME POR BANCO:
--------------------------------------------------------------------------------
              Volume Total  Qtd Registros  % do Total
bank_name                                            
Banco Hidra  41,092,024.89           2860       48.13
Banco Shield 44,293,882.66           3085       51.87


---

# 4. KPI 2 & 3: Market Share e Contratos Ativos

In [15]:
# KPI 2: Market Share
print("=" * 80)
print("KPI 2: MARKET SHARE")
print("=" * 80)

bancos = df['bank_name'].unique()
for banco in bancos:
    share = ShieldMetrics.market_share(df, banco)
    print(f"\n{banco:<25}: {share:>6.2f}%")

# KPI 3: Contratos Ativos
print("\n\n" + "=" * 80)
print("KPI 3: CONTRATOS ATIVOS")
print("=" * 80)

contratos_total = ShieldMetrics.contratos_ativos(df)
print(f"\nTotal de Contratos: {contratos_total:,}")

# Por banco
print("\nCONTRATOS POR BANCO:")
contratos_por_banco = df.groupby('bank_name')['contract_id'].count()
for banco, qtd in contratos_por_banco.items():
    pct = (qtd / contratos_total) * 100
    print(f"{banco:<25}: {qtd:>10,} ({pct:>5.2f}%)")


KPI 2: MARKET SHARE

Banco Shield             :  51.87%

Banco Hidra              :  48.13%


KPI 3: CONTRATOS ATIVOS

Total de Contratos: 5,945

CONTRATOS POR BANCO:
Banco Hidra              :      2,860 (48.11%)
Banco Shield             :      3,085 (51.89%)


---

# 5. KPI 4: Nível de Risco Médio

In [16]:
# KPI 4: Nível de Risco Médio
risco_total = ShieldMetrics.risco_medio(df)

print("=" * 80)
print("KPI 4: NÍVEL DE RISCO MÉDIO")
print("=" * 80)
print(f"\nRisco Médio Geral: {risco_total:.2f}%")

# Classificação de risco
if risco_total < 3:
    classificacao = "🟢 BAIXO"
elif risco_total < 7:
    classificacao = "🟡 MODERADO"
elif risco_total < 12:
    classificacao = "🔴 ELEVADO"
else:
    classificacao = "⚠️  CRÍTICO"

print(f"Classificação: {classificacao}")

# Por banco
print("\nRISCO MÉDIO POR BANCO:")
risco_por_banco = df.groupby('bank_name')['risk_score'].apply(lambda x: x.mean() * 100)
for banco, risco in risco_por_banco.items():
    if risco < 3:
        clf = "🟢 Baixo"
    elif risco < 7:
        clf = "🟡 Moderado"
    elif risco < 12:
        clf = "🔴 Elevado"
    else:
        clf = "⚠️  Crítico"
    print(f"{banco:<25}: {risco:>6.2f}% ({clf})")

# Contratos em alto risco
contratos_alto_risco = (df['is_high_risk'] == 1).sum()
pct_alto_risco = (contratos_alto_risco / len(df)) * 100
print(f"\nContratos em Alto Risco (DPD ≥ 30): {contratos_alto_risco:,} ({pct_alto_risco:.2f}%)")


KPI 4: NÍVEL DE RISCO MÉDIO

Risco Médio Geral: 11.46%
Classificação: 🔴 ELEVADO

RISCO MÉDIO POR BANCO:
Banco Hidra              :  14.06% (⚠️  Crítico)
Banco Shield             :   9.05% (🔴 Elevado)

Contratos em Alto Risco (DPD ≥ 30): 59 (0.99%)


---

# 6. KPI 5: Volume por Tempo (Série Temporal)

In [17]:
# KPI 5: Volume por Tempo
df_tempo = ShieldMetrics.volume_tempo(df)

print("=" * 80)
print("KPI 5: VOLUME POR PERÍODO")
print("=" * 80)
print(f"\nPeríodos disponíveis: {df_tempo['ano_mes'].nunique()}")
print(f"Intervalo: {df_tempo['ano_mes'].min()} a {df_tempo['ano_mes'].max()}")

print("\n" + df_tempo.to_string(index=False))



KPI 5: VOLUME POR PERÍODO

Períodos disponíveis: 13
Intervalo: 2025-01 a 2025-13

ano_mes    bank_name  financed_amount
2025-01  Banco Hidra     3,052,246.38
2025-01 Banco Shield     3,554,721.19
2025-02  Banco Hidra     3,293,360.73
2025-02 Banco Shield     3,511,872.73
2025-03  Banco Hidra     3,447,647.62
2025-03 Banco Shield     3,931,393.97
2025-04  Banco Hidra     3,315,137.94
2025-04 Banco Shield     3,568,093.19
2025-05  Banco Hidra     3,231,913.83
2025-05 Banco Shield     3,423,091.88
2025-06  Banco Hidra     3,358,512.05
2025-06 Banco Shield     3,443,097.74
2025-07  Banco Hidra     3,546,748.90
2025-07 Banco Shield     4,168,408.49
2025-08  Banco Hidra     3,260,104.62
2025-08 Banco Shield     3,781,566.39
2025-09  Banco Hidra     4,091,838.42
2025-09 Banco Shield     3,789,224.34
2025-10  Banco Hidra     3,478,167.79
2025-10 Banco Shield     3,561,653.61
2025-11  Banco Hidra     3,505,484.40
2025-11 Banco Shield     3,682,539.77
2025-12  Banco Hidra     3,361,090.64
2025-1

---

# 7. KPI 6: Share por Produto

In [18]:
# KPI 6: Share por Produto
df_produtos = ShieldMetrics.share_por_produto(df)

print("=" * 80)
print("KPI 6: SHARE POR PRODUTO")
print("=" * 80)
print(f"\nProdutos identificados: {df['product_name'].nunique()}")
print("\n" + df_produtos.to_string(index=False))



KPI 6: SHARE POR PRODUTO

Produtos identificados: 20

                  product_name    bank_name  financed_amount
                 Cartão Spider  Banco Hidra       111,873.83
                 Cartão Spider Banco Shield       116,474.16
              Cartão Vibranium  Banco Hidra             0.00
              Cartão Vibranium Banco Shield             0.00
               Consórcio Stark  Banco Hidra     1,624,425.49
               Consórcio Stark Banco Shield     1,734,189.32
              Consórcio Xandar  Banco Hidra       695,066.05
              Consórcio Xandar Banco Shield       837,232.08
          Conta Digital SHIELD  Banco Hidra             0.00
          Conta Digital SHIELD Banco Shield             0.00
             Crédito Kamar-Taj  Banco Hidra     5,216,345.19
             Crédito Kamar-Taj Banco Shield     5,103,088.80
               Crédito Pantera  Banco Hidra             0.00
               Crédito Pantera Banco Shield             0.00
             Crédito Wakandano 

---

# 8. KPI 7: Análise Regional (Domínio Geográfico)

In [19]:
# KPI 7: Análise Regional
df_regioes = ShieldMetrics.group_region(df)

print("=" * 80)
print("KPI 7: DOMÍNIO GEOGRÁFICO (VOLUME POR REGIÃO)")
print("=" * 80)
print(f"\nRegiões identificadas: {df['location_name'].nunique()}")
print(f"Macro-regiões: {df['macro_region'].nunique()}")

print("\n" + df_regioes.sort_values('financed_amount', ascending=False).to_string(index=False))



KPI 7: DOMÍNIO GEOGRÁFICO (VOLUME POR REGIÃO)

Regiões identificadas: 15
Macro-regiões: 5

 location_name    bank_name  financed_amount
Rio de Janeiro Banco Shield     3,456,479.27
        Recife Banco Shield     3,251,752.24
Belo Horizonte Banco Shield     3,234,494.49
     Nova York Banco Shield     3,191,557.72
        Xandar  Banco Hidra     3,073,595.60
    Sanctum SP  Banco Hidra     3,057,661.86
Rio de Janeiro  Banco Hidra     3,027,274.72
    Sanctum SP Banco Shield     3,022,988.02
      Salvador Banco Shield     3,009,854.27
       Wakanda Banco Shield     2,984,718.67
      Knowhere Banco Shield     2,979,679.51
      Curitiba  Banco Hidra     2,959,655.47
       Sokovia Banco Shield     2,950,325.61
      Curitiba Banco Shield     2,941,858.03
      Brasília  Banco Hidra     2,897,592.14
        Recife  Banco Hidra     2,889,984.58
Belo Horizonte  Banco Hidra     2,877,528.35
     São Paulo Banco Shield     2,788,200.23
  Porto Alegre  Banco Hidra     2,782,953.54
     Nova

---

# 9. KPI 8: Risco vs Volume vs Contratos (Scatter Plot)

In [20]:
# KPI 8: Risco vs Volume vs Contratos
df_risk = ShieldMetrics.risco_volume_contratos(df)

print("=" * 80)
print("KPI 8: AGREGAÇÃO RISCO, VOLUME E CONTRATOS")
print("=" * 80)
print("\n" + df_risk.to_string(index=False))


# Estatísticas por banco
print("\n" + "=" * 80)
print("RESUMO POR BANCO:")
print("=" * 80)
print(df_risk.groupby('bank_name')[['financed_amount', 'risk_score', 'contract_id']].agg({
    'financed_amount': ['sum', 'mean'],
    'risk_score': ['mean', 'max', 'min'],
    'contract_id': 'sum'
}).round(4))


KPI 8: AGREGAÇÃO RISCO, VOLUME E CONTRATOS

                  product_name    bank_name  financed_amount  risk_score  contract_id
                 Cartão Spider  Banco Hidra       111,873.83        0.14          145
                 Cartão Spider Banco Shield       116,474.16        0.09          145
              Cartão Vibranium  Banco Hidra             0.00        0.15          128
              Cartão Vibranium Banco Shield             0.00        0.09          146
               Consórcio Stark  Banco Hidra     1,624,425.49        0.14          139
               Consórcio Stark Banco Shield     1,734,189.32        0.09          149
              Consórcio Xandar  Banco Hidra       695,066.05        0.15          142
              Consórcio Xandar Banco Shield       837,232.08        0.09          166
          Conta Digital SHIELD  Banco Hidra             0.00        0.14          136
          Conta Digital SHIELD Banco Shield             0.00        0.09          153
          

---

# 10. Insights e Conclusões

## 📈 Principais Insights

### Sobre o Mercado
- **Competição Intensa:** A disputa de mercado entre Banco Shield e Hidra Bank é bastante acirrada
- **Volatilidade**: O volume por período demonstra flutuações significativas
- **Distribuição Geográfica:** Algumas regiões têm maior concentração de volume

### Sobre o Risco
- **Nível de Risco Moderado:** O risco médio está dentro de faixas aceitáveis
- **Contratos em Atraso:** Uma porcentagem dos contratos apresenta 30+ dias em atraso
- **Variação por Produto:** Diferentes produtos têm perfis de risco distintos

### Sobre os Produtos
- **Diversificação:** Os produtos têm diferentes prazos e taxas
- **Volume Distribuído:** Cada produto tem seu mercado específico
- **Relação Prazo-Taxa:** Produtos com prazos maiores tendem a ter taxas diferentes



## 📚 Dicionário de Dados Disponível

Para documentação completa sobre todas as colunas, veja **DOCUMENTATION.md** no repositório!